# Working with uDALES Facet Data

This tutorial covers how to work with uDALES facet data, including surface quantities such as shear stress, surface pressure, surface energy balance terms, and temperature inside each of the facets.

The **UDBase** class provides comprehensive methods for facet analysis:
- **calculate_frontal_properties**: Calculate skylines, frontal areas, blockage ratios
- **plot_fac_type**: Display the type of surface for each facet
- **assign_prop_to_fac**: Assign properties to facets for calculation/visualization
- **plot_fac**: Display a surface variable on the mesh
- **load_fac_momentum**: Load facet pressure and shear stresses
- **load_fac_eb**: Load surface energy balance terms
- **load_seb**: Load all surface energy balance terms
- **load_fac_temperature**: Load temperatures inside facets
- **area_average_seb**: Area-averaged surface energy balance
- **area_average_fac**: Area-averaging over facets
- **time_average**: Time-averaging method (static)
- **convert_fac_to_field**: Convert facet data to 3D density field

## 1. Import Libraries and Setup

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Add uDALES python tools to path
udales_root = Path.cwd().parent.parent if 'docs' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(udales_root / 'tools' / 'python'))

from udbase import UDBase

# Initialize with experiment containing facet data
expnr = '065'
expdir = Path('../experiments/065')

if expdir.exists():
    sim = UDBase(case_dir=str(expdir))
    print(f"✓ Initialized UDBase for experiment {expnr}")
else:
    print(f"⚠ Directory {expdir} not found - adjust path as needed")
    sim = None

## 2. Calculate Frontal Properties

Calculate skyline heights, frontal areas, and blockage ratios in x- and y-directions.

In [ ]:
if sim is not None and sim.geom is not None:
    try:
        # Calculate frontal properties
        res = sim.calculate_frontal_properties()
        
        print("Frontal properties calculated:")
        print(f"\nX-direction:")
        print(f"  Frontal area: {res['frontal_area_x']:.1f} m²")
        print(f"  Blockage ratio: {res['blockage_ratio_x']:.3f}")
        print(f"\nY-direction:")
        print(f"  Frontal area: {res['frontal_area_y']:.1f} m²")
        print(f"  Blockage ratio: {res['blockage_ratio_y']:.3f}")
        print()
        print("Frontal area = projected surface area summed over buildings")
        print("Blockage ratio = fraction of cross-section occupied by buildings")
        
    except Exception as e:
        print(f"Error: {e}")
else:
    print("Example output (when geometry is available):")
    print("  Frontal area_x: 1250.0 m²")
    print("  Blockage ratio_x: 0.195")

## 3. Facet Types and Properties

Facet types contain properties for each wall type, including physical and thermal properties.

In [ ]:
if sim is not None and hasattr(sim, 'factypes') and sim.factypes is not None:
    print("Facet type properties:")
    print("  - id: Type identifier")
    print("  - name: Type name (wall/roof/ground)")
    print("  - lGR: Evaporation enabled (boolean)")
    print("  - z0: Momentum roughness length [m]")
    print("  - z0h: Heat roughness length [m]")
    print("  - al: Surface albedo [-]")
    print("  - em: Surface emissivity [-]")
    print("  - d: Layer thickness [m]")
    print("  - C: Volumetric heat capacity [J/m³K]")
    print("  - lam: Thermal conductivity [W/mK]")
    print()
    print(f"Number of facet types: {len(sim.factypes['id'])}")
else:
    print("Facet type data contains physical/thermal properties")
    print("Example: albedo, emissivity, roughness lengths, thermal properties")

## 4. Key Facet Methods Overview

### Visualization Methods
- `plot_fac_type()` - Display surface types (walls/roofs/ground)
- `plot_fac(data, title, cmap)` - Display variable on 3D mesh
- `assign_prop_to_fac(prop)` - Assign properties to individual facets

### Data Loading Methods
- `load_fac_momentum(var, time)` - Pressure, shear stress (fac.expnr.nc)
- `load_fac_eb(var, time)` - Individual SEB terms (facEB.expnr.nc)
- `load_seb(time)` - All SEB terms at once (recommended)
- `load_fac_temperature(time)` - Facet temperature (facT.expnr.nc)

### Analysis Methods
- `area_average_fac(data, areas)` - Area-weighted averaging
- `area_average_seb(seb_dict, facet_type)` - Average SEB by type
- `time_average(data, axis)` - Time averaging (static method)
- `convert_fac_to_field(fac_data, method)` - Convert to 3D field

## 5. Load Surface Energy Balance (SEB) Data

The `load_seb()` method loads all energy balance terms at once - recommended approach.

In [ ]:
# Example: Load SEB data
print("Usage:")
print("  seb = sim.load_seb(time=3600)")
print()
print("Returns dictionary with:")
print("  seb['qsens'] : Sensible heat flux [W/m²]")
print("  seb['qlat']  : Latent heat flux [W/m²]")
print("  seb['qnet']  : Net radiation [W/m²]")
print("  seb['qstor'] : Storage heat flux [W/m²]")
print("  seb['area']  : Facet areas [m²]")
print("  seb['time']  : Output time [s]")
print()
print("Energy balance:")
print("  qnet = qsens + qlat + qstor")
print()
print("Example analysis:")
print("""
# Load SEB at t=3600s
seb = sim.load_seb(time=3600)

# Check energy balance
balance = seb['qnet'] - (seb['qsens'] + seb['qlat'] + seb['qstor'])
print(f'Max residual: {np.abs(balance).max():.2f} W/m²')

# Visualize sensible heat flux
sim.plot_fac(seb['qsens'], title='Sensible Heat Flux [W/m²]', cmap='RdBu_r')
""")

## 6. Area-Averaged Surface Energy Balance

Calculate area-weighted averages for all SEB components, optionally by facet type.

In [ ]:
# Example: Area-average SEB
print("Usage:")
print("  seb = sim.load_seb(time=3600)")
print()
print("  # Average over all facets")
print("  avg_all = sim.area_average_seb(seb)")
print()
print("  # Average by facet type")
print("  avg_wall = sim.area_average_seb(seb, facet_type='wall')")
print("  avg_roof = sim.area_average_seb(seb, facet_type='roof')")
print("  avg_ground = sim.area_average_seb(seb, facet_type='ground')")
print()
print("Returns dictionary:")
print("  {'qsens': ..., 'qlat': ..., 'qnet': ..., 'qstor': ...}")
print()
print("Complete example:")
print("""
seb = sim.load_seb(time=3600)

# Calculate type-specific averages
types = ['wall', 'roof', 'ground']
for ftype in types:
    avg = sim.area_average_seb(seb, facet_type=ftype)
    print(f'{ftype.capitalize()}:')
    print(f'  Qsens: {avg["qsens"]:6.2f} W/m²')
    print(f'  Qlat:  {avg["qlat"]:6.2f} W/m²')
    print(f'  Qnet:  {avg["qnet"]:6.2f} W/m²')
    print(f'  Qstor: {avg["qstor"]:6.2f} W/m²')
""")

## 7. Complete Workflow Example

Typical facet analysis workflow from data loading to visualization and analysis.

In [ ]:
# Complete facet analysis workflow
print("Complete workflow:")
print("""
# 1. Initialize
sim = UDBase(case_dir='./experiment')

# 2. Calculate urban metrics
frontal = sim.calculate_frontal_properties()
print(f'Blockage ratio: {frontal["blockage_ratio_x"]:.3f}')

# 3. Load SEB data
seb = sim.load_seb(time=3600)

# 4. Calculate averages by type
avg_all = sim.area_average_seb(seb)
avg_wall = sim.area_average_seb(seb, facet_type='wall')
avg_roof = sim.area_average_seb(seb, facet_type='roof')
avg_ground = sim.area_average_seb(seb, facet_type='ground')

# 5. Check energy balance
balance = avg_all['qnet'] - (avg_all['qsens'] + 
                              avg_all['qlat'] + 
                              avg_all['qstor'])
print(f'Energy balance residual: {balance:.2f} W/m²')

# 6. Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
sim.plot_fac(seb['qsens'], title='Sensible Heat Flux', ax=axes[0,0])
sim.plot_fac(seb['qlat'], title='Latent Heat Flux', ax=axes[0,1])
sim.plot_fac(seb['qnet'], title='Net Radiation', ax=axes[1,0])
sim.plot_fac(seb['qstor'], title='Storage Flux', ax=axes[1,1])

# 7. Time series analysis (if multiple times)
seb_all = sim.load_seb()  # All times
qsens_avg = [sim.area_average_seb(seb_all[t])['qsens'] 
             for t in range(len(seb_all['time']))]
plt.plot(seb_all['time'], qsens_avg)
plt.xlabel('Time [s]')
plt.ylabel('⟨Qsens⟩ [W/m²]')
""")

## 8. Summary

### Key Capabilities
- ✅ Load all facet data types (momentum, energy balance, temperature)
- ✅ Area-weighted averaging over facets with type-specific analysis
- ✅ 3D visualization of facet variables on building surfaces
- ✅ Convert facet data to 3D fields for volume analysis
- ✅ Calculate urban canopy metrics (frontal area, blockage ratio)

### Facet Data Sources
| Method | File | Variables |
|--------|------|-----------|
| `load_fac_momentum` | fac.expnr.nc | pres, tau_x/y/z, htc |
| `load_fac_eb` | facEB.expnr.nc | qsens, qlat, qnet, qstor |
| `load_seb` | facEB.expnr.nc | All SEB terms (recommended) |
| `load_fac_temperature` | facT.expnr.nc | Temperature profiles |

### Next Steps
- **geometry_tutorial.ipynb** - Creating and manipulating urban geometries
- `tools/python/facets_example.py` - Complete working examples
- `tools/python/advanced_analysis_example.py` - Advanced workflows
- uDALES documentation: https://u-dales.readthedocs.io/